Data Preprocessing

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('train.csv')
df_raw = pd.read_csv('train.csv')

# drop unnecessary columns
df = df.drop(columns=['weather_temperature', 'weather_wind_mph', 'weather_humidity', 'weather_detail', 'stadium'])
df.head()

# first add total_score column
df['total_score'] = df['score_home'] + df['score_away']


# add column that represented current record for each team before each game of a season
# ensure games are sorted chronologically within each season
df["datetime"] = pd.to_datetime(df["schedule_date"])
df = df.sort_values(["schedule_season", "datetime"]).reset_index(drop=True)

# make output lists
home_records = []
away_records = []

# make dictionaries to track each team's W-L-T within the current season
team_wins = {}
team_losses = {}
team_ties = {}

current_season = None

for i, row in df.iterrows():
    season = row["schedule_season"]
    home = row["team_home"]
    away = row["team_away"]
    home_score = row["score_home"]
    away_score = row["score_away"]
   
    # new season, we reset all
    if season != current_season:
        team_wins = {}
        team_losses = {}
        team_ties = {}
        current_season = season

    # initialize teams for this season if needed
    for team in [home, away]:
        if team not in team_wins:
            team_wins[team] = 0
            team_losses[team] = 0
            team_ties[team] = 0

    # add current record before the game
    home_records.append(
        f"{team_wins[home]}-{team_losses[home]}-{team_ties[home]}"
    )
    away_records.append(
        f"{team_wins[away]}-{team_losses[away]}-{team_ties[away]}"
    )

    # update records after the game
    if home_score > away_score:
        team_wins[home] += 1
        team_losses[away] += 1
    elif away_score > home_score:
        team_wins[away] += 1
        team_losses[home] += 1
    else:
        # tie
        team_ties[home] += 1
        team_ties[away] += 1

# add results to dataframe
df["home_team_record"] = home_records
df["away_team_record"] = away_records


# make individual columns for wins, losses, and ties
df['home_wins'] = df['home_team_record'].apply(lambda x: int(x.split('-')[0]))
df['home_losses'] = df['home_team_record'].apply(lambda x: int(x.split('-')[1]))
df['home_ties'] = df['home_team_record'].apply(lambda x: int(x.split('-')[2]))
df['away_wins'] = df['away_team_record'].apply(lambda x: int(x.split('-')[0]))
df['away_losses'] = df['away_team_record'].apply(lambda x: int(x.split('-')[1]))
df['away_ties'] = df['away_team_record'].apply(lambda x: int(x.split('-')[2]))


# filter games that have already been recorded, no scheduled games
df = df[df["datetime"] <= "2025-11-04"]

In [2]:
# 1. Compute each team's average score per season
team_season_avg = (
    df.groupby(["team_home", "schedule_season"])["score_home"].mean().reset_index()
)
team_season_avg.columns = ["team", "season", "avg_score"]

# also include away team scoring
team_season_avg_away = (
    df.groupby(["team_away", "schedule_season"])["score_away"].mean().reset_index()
)
team_season_avg_away.columns = ["team", "season", "avg_score"]

# combine home + away scoring for a true team season average
team_season_avg = pd.concat([team_season_avg, team_season_avg_away])
team_season_avg = team_season_avg.groupby(["team", "season"])["avg_score"].mean().reset_index()

# 2. Shift averages to represent "previous season"
team_season_avg["prev_season"] = team_season_avg["season"] + 1

# this means: prev_season avg is used in the next year's games
team_prev = team_season_avg[["team", "prev_season", "avg_score"]]
team_prev.columns = ["team", "schedule_season", "prev_season_avg"]

# 3. Merge into main df
df = df.merge(team_prev, left_on=["team_home", "schedule_season"], right_on=["team", "schedule_season"], how="left")
df.rename(columns={"prev_season_avg": "home_prev_avg"}, inplace=True)
df = df.drop(columns=["team"])

df = df.merge(team_prev, left_on=["team_away", "schedule_season"], right_on=["team", "schedule_season"], how="left")
df.rename(columns={"prev_season_avg": "away_prev_avg"}, inplace=True)
df = df.drop(columns=["team"])

In [4]:
home = df[["schedule_season", "datetime", "team_home", "score_home", "score_away"]].rename(
    columns={"team_home": "team", "score_home": "points_scored", "score_away": "points_allowed"}
)

away = df[["schedule_season", "datetime", "team_away", "score_away", "score_home"]].rename(
    columns={"team_away": "team", "score_away": "points_scored", "score_home": "points_allowed"}
)

long_df = pd.concat([home, away])
long_df = long_df.sort_values(["team", "schedule_season", "datetime"]).reset_index(drop=True)

groups = long_df.groupby(["team", "schedule_season"])

long_df["rolling_scored"] = groups["points_scored"].transform(
    lambda s: s.shift().expanding().mean()
)

long_df["rolling_allowed"] = groups["points_allowed"].transform(
    lambda s: s.shift().expanding().mean()
)

df = df.merge(
    long_df[["team", "schedule_season", "datetime", "rolling_scored", "rolling_allowed"]],
    left_on=["team_home", "schedule_season", "datetime"],
    right_on=["team", "schedule_season", "datetime"],
    how="left"
).rename(
    columns={
        "rolling_scored": "home_rolling_scored",
        "rolling_allowed": "home_rolling_allowed"
    }
).drop(columns=["team"])


df = df.merge(
    long_df[["team", "schedule_season", "datetime", "rolling_scored", "rolling_allowed"]],
    left_on=["team_away", "schedule_season", "datetime"],
    right_on=["team", "schedule_season", "datetime"],
    how="left"
).rename(
    columns={
        "rolling_scored": "away_rolling_scored",
        "rolling_allowed": "away_rolling_allowed"
    }
).drop(columns=["team"])

- Avg points scored for each team
- Avg points given up for each team
- Team record
- 

In [5]:
df.describe

<bound method NDFrame.describe of      schedule_date  schedule_season schedule_week  schedule_playoff  \
0         9/8/2005             2005             1             False   
1        9/11/2005             2005             1             False   
2        9/11/2005             2005             1             False   
3        9/11/2005             2005             1             False   
4        9/11/2005             2005             1             False   
...            ...              ...           ...               ...   
4443     11/2/2025             2025             9             False   
4444     11/2/2025             2025             9             False   
4445     11/2/2025             2025             9             False   
4446     11/3/2025             2025             9             False   
4447     11/4/2025             2025             9             False   

                  team_home  score_home  score_away             team_away  \
0      New England Patriots        3